# Bước 2: Basket Preparation for Association Analysis

Notebook này thực hiện bước chuyển đổi dữ liệu giao dịch từ dữ liệu đã làm sạch thành định dạng giỏ hàng (basket format) thích hợp để áp dụng các thuật toán khai thác luật kết hợp như Apriori và FP-Growth trong các bước tiếp theo.

## Mục tiêu

1. Đọc dữ liệu giao dịch đã làm sạch

2. Biến đổi dữ liệu thành ma trận Invoice × Product

3. Mã hoá ma trận thành dạng boolean (0/1)

4. Kiểm tra phân phối số lượng sản phẩm trên mỗi hóa đơn

5. Lưu ma trận boolean ra file để sử dụng cho mô hình mining luật kết hợp


Các thuật toán khai phá luật kết hợp không làm việc với dữ liệu dạng transactional gốc theo từng dòng, mà yêu cầu dữ liệu ở dạng:

        Product A | Product B | Product C | ...
Invoice 1 1 | 0 | 1 | ...
Invoice 2 0 | 1 | 1 | ...
Invoice 3 1 | 1 | 0 | ...

Trong đó:

- `1` nghĩa là sản phẩm có trong giỏ hàng
- `0` nghĩa là không có

Việc chuẩn bị dữ liệu dạng basket là chìa khóa quan trọng trước khi áp dụng Apriori / FP-Growth.


In [2]:
# PARAMETERS (for papermill)

# File dữ liệu đã làm sạch từ bước 1
CLEANED_DATA_PATH = "data/processed/cleaned_uk_data.csv"

# Đường dẫn lưu basket_bool dạng parquet 
BASKET_BOOL_PATH = "data/processed/basket_bool.parquet"

# Tên cột trong dữ liệu đã làm sạch
INVOICE_COL = "InvoiceNo"
ITEM_COL = "Description"
QUANTITY_COL = "Quantity"

# Ngưỡng để coi một item là "có trong giỏ"
# (Quantity >= THRESHOLD -> 1, ngược lại 0)
THRESHOLD = 1


## Set up

In [3]:
%load_ext autoreload
%autoreload 2

import os
import sys
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


# Determine correct project root
cwd = os.getcwd()
if os.path.basename(cwd) == "notebooks":
    project_root = os.path.abspath("..")
else:
    project_root = cwd

src_path = os.path.join(project_root, "src")
if src_path not in sys.path:
    sys.path.append(src_path)


from apriori_library import BasketPreparer


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### Tải dữ liệu đã làm sạch

In [4]:
# Đọc dữ liệu đã làm sạch từ Notebook 01
df_clean = pd.read_csv(CLEANED_DATA_PATH, parse_dates=["InvoiceDate"])

print("Thông tin dữ liệu đã làm sạch:")
print(f"- Số giao dịch: {df_clean.shape[0]:,}")
print(f"- Số cột: {df_clean.shape[1]}")
print(f"- Số hoá đơn (InvoiceNo) duy nhất: {df_clean['InvoiceNo'].nunique():,}")
print(f"- Số sản phẩm (Description) duy nhất: {df_clean['Description'].nunique():,}")

df_clean.head()


FileNotFoundError: [Errno 2] No such file or directory: 'data/processed/cleaned_uk_data.csv'

### Khởi tạo BasketPreparer và tạo basket

In [ ]:
# Khởi tạo BasketPreparer
basket_maker = BasketPreparer(
    df=df_clean,
    invoice_col=INVOICE_COL,
    item_col=ITEM_COL,
    quantity_col=QUANTITY_COL,
)

# Tạo basket (Invoice x Item, giá trị = tổng Quantity)
basket = basket_maker.create_basket()

print("Kích thước basket (ma trận Invoice x Item):")
print(f"- Số hoá đơn (rows): {basket.shape[0]:,}")
print(f"- Số sản phẩm (columns): {basket.shape[1]:,}")

basket.iloc[:5, :10]  # xem thử 5 hoá đơn đầu, 10 sản phẩm đầu


### Encode basket thành 0/1

Tỉ lệ ô = 1 càng thấp chứng tỏ ma trận càng thưa (sparse), phù hợp với các thuật toán luật kết hợp.

In [ ]:
# Mã hoá basket thành dạng boolean (0/1) với ngưỡng THRESHOLD
basket_bool = basket_maker.encode_basket(threshold=THRESHOLD)

print("Thông tin basket_bool:")
print(f"- Kích thước: {basket_bool.shape[0]:,} x {basket_bool.shape[1]:,}")
print(f"- Tỉ lệ ô = 1 (mua hàng): {basket_bool.values.mean():.4f}")

basket_bool.iloc[:5, :10]


In [ ]:
# Phân phối số lượng item trong mỗi giỏ (số sản phẩm mỗi hoá đơn)
items_per_invoice = basket_bool.sum(axis=1)

print("Số sản phẩm trung bình trong mỗi hoá đơn:")
print(f"- Mean: {items_per_invoice.mean():.2f}")
print(f"- Median: {items_per_invoice.median():.0f}")
print(f"- Max: {items_per_invoice.max():,}")

items_per_invoice.describe()


In [ ]:
# Lưu basket_bool vào file parquet để dùng cho Apriori ở Notebook 03
basket_maker.save_basket_bool(BASKET_BOOL_PATH)

print("Đã lưu basket_bool thành công:")
print(f"- File: {BASKET_BOOL_PATH}")
print(f"- Kích thước: {basket_bool.shape[0]:,} x {basket_bool.shape[1]:,}")
